In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import time

import numpy as np
import dask
import GPy
from elfi import *
from elfi.examples.ma2 import *
from elfi.storage import DictListStore
from distributed import Client
from dask.dot import dot_graph
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

#matplotlib.style.use('ggplot')
#%matplotlib inline

import logging
elfi_log = logging.getLogger("elfi")
elfi_log.setLevel(20)
elfi_log.propagate = False
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s - %(message)s')
ch.setFormatter(formatter)
elfi_log.addHandler(ch)

In [ ]:
n = 1000
t1 = 0.6
t2 = 0.2

# Set up observed data y
latents = np.random.randn(n+2)
n_sim = 1
y = MA2(n, t1, t2, n_sim, latents=latents)

# Plot
if False:
    plt.figure(figsize=(11, 6))
    plt.plot(np.arange(0,n),y[0,:])
    plt.scatter(np.arange(-2,n), latents);

In [ ]:
# Set up the simulator
simulator = partial(MA2, n)

# Set up autocovariance summaries
ac1 = partial(autocov, 1)
ac2 = partial(autocov, 2)

# Specify the graphical model
t1 = Prior('t1', 'uniform', 0, 1)
#t2 = Prior('t2', 'uniform', 0, 1)
Y = Simulator('MA2', simulator, t1, t2, observed=y)
S1 = Summary('S1', ac1, Y)
S2 = Summary('S2', ac2, Y)
d = Discrepancy('d', distance, S1, S2)

# Specify the number of simulations
n_sim = 60
n_batch = 4

In [ ]:
# Used for inspecting inference process
def plot_bolfi_process(store):
    fig = plt.figure(figsize=(5, 10))
    t = (2,5,10,20,60)
    n_fig = len(t)
    for i in range(n_fig):
        ax = fig.add_subplot(n_fig,1,i+1)
        f = store.get("BOLFI-model", t[i])[0].gp.plot(ax=ax)
        ax.legend_.remove()
        ax.set_title("Discrepancy model after {} samples".format(t[i]))
        ax.set_xlim(0, 1)
        GPy.plotting.show(f)
    plt.tight_layout()
    plt.show()

In [ ]:
store = DictListStore()
gp_model = GPyModel(input_dim=1, bounds=((0,1),), kernel_var=0.3, kernel_scale=0.2, noise_var=0.01)
bolfi = BOLFI(d, [t1], batch_size=n_batch, n_surrogate_samples=n_sim, model=gp_model, store=store)
post = bolfi.infer()
fig = bolfi.model.gp.plot()
GPy.plotting.show(fig)
post.plot()

In [ ]:
plot_bolfi_process(store)

In [ ]:
store = DictListStore()
gp_model = GPyModel(input_dim=1, bounds=((0,1),), kernel_var=0.3, kernel_scale=0.2, noise_var=0.01)
acq = AsyncBolfiAcquisition(model=gp_model, n_samples=n_sim, rbf_scale=0.1)
async_bolfi = BOLFI(d, [t1], batch_size=n_batch, model=gp_model, acquisition=acq, store=store, sync=False)
async_post = async_bolfi.infer()
fig = async_bolfi.model.gp.plot()
GPy.plotting.show(fig)
async_post.plot()

In [ ]:
plot_bolfi_process(store)

In [ ]:
from IPython.display import display
display(bolfi.model.gp)
display(async_bolfi.model.gp)